In [38]:
import numpy as np
import openrouteservice
from openrouteservice.geocode import pelias_search
import xarray as xr
from datetime import datetime
from datetime import timedelta
from datetime import date as ddate

ORS_API_KEY = "5b3ce3597851110001cf62485c01b629a61e4f57b5449ab3a0000ec5"

def get_temp_series(location = "Bergheimer Straße 116, 69115 Heidelberg, Germany"):
    """
    Given a location returns an xarray dataset of hourly temperatures in Kelvin
    INPUTS:
        something
    OUTPUTS:
        ds (xarray dataset): has coordinates [longitude (float32), latitude (float32), time (datetime64)]
        
    TODO: specify time bounds, properly comment
    """
    clnt = openrouteservice.Client(key=ORS_API_KEY)
    call = pelias_search(clnt, location)
    for feature in call["features"]:
        geom = feature["geometry"]["coordinates"]

    fn = "temp_data.nc"
    ds = xr.open_dataset(fn)
    #yearmonth = "2021-10-31"
    #yearmonth = datetime.strptime(yearmonth, '%Y-%m-%d')
    ds = ds.sel(latitude=round(geom[1])).sel(longitude=round(geom[0]))

    # interpolate missing hours of the day (sometimes data only from 06h to 22h)
    ds = ds.resample(time="1h").interpolate("linear")
    return ds

def get_month_endpoints(date_str, include_midnight = True):
    """
    Given a year-month string, return the endpoints required for the timeslice to select temperatures of a given month
    INPUTS:
        date_str (str): should be 'year-month', ex. '2020-02'
        include_midnight (bool, optional): whether or not to include first day of next month at midnight. Needed for
            integration approach to include 11pm to midnight of last day in the integration
    OUTPUTS:
        start_str (str): string for first day of month, ex. '2020-02-01'
        end_str (str): if include_midnight = True, string for first day of next month at midnight, ex. '2020-03-01-00'
                       if include_midnight = False, string for last day of month at 11pm, ex. '2020-03-31-23'
    """
    start_year , start_month = date_str.split('-')
    assert (int(start_month) <= 12), 'Error in get_month_endpoints: month can not be >12!'
    start_str = start_year + '-' + start_month + '-01'
    
    if include_midnight:
        if int(start_month) < 12:
            if include_midnight:
                end_month = '{:02d}'.format(int(start_month)+1)
            else:
                end_month = start_month
            end_year = start_year
        else: # start_month = 12
            end_month = '01'
            end_year = str(int(start_year)+1)
        end_str = end_year + '-' + end_month + '-01-00'
    else:
        end_month = '{:02d}'.format(int(start_month))
        end_year = start_year
        # The day 28 exists in every month. 4 days later, it's always next month
        next_month = ddate(int(start_year), int(start_month), 1).replace(day=28) + timedelta(days=4)
        # subtracting the number of the current day brings us back to the last dayone month
        end_str = next_month - timedelta(days=next_month.day)
        end_str = str(end_str) + '-23'

    return start_str, end_str

def calc_degreedays_int(type, date_str, Tref = None):
    """
    Calculate the degree days for a given month using the integration approach (higher fidelity). 
    For futher details see for example
        Day, & Karayiannis, T. G. (1998). Degree-days: Comparison of calculation methods. 
        Building Services Engineering Research & Technology, 19(1), 7-13. https://doi.org/10.1177/014362449801900102
    INPUTS:
        type (str): either 'heating' for HDD or 'cooling' for CDD
        date_str (str): should be 'year-month', ex. '2020-02'
        Tref (float, optional): reference or base temperature. Default is 15.5C for HDD, 22C for CDD (standard values)
    OUTPUTS:
        dd['t2m'].values (float): Cumulative degree days for the given month
    """
    if type not in ('heating','cooling'):
        print("WARNING calc_degreedays_int: degree days type not understood. Defaulting to type='heating'.")
        type = 'heating'
    
    ds = get_temp_series() #TODO specify location here
    # this takes from midnight of first day to midnight of first day of following month
    month_ds = ds.sel(time=slice(*get_month_endpoints(date_str, include_midnight=True)))

    if type == 'heating':
        if Tref != None: Tref = 288.65 # (15.5 C)
        month_ds['t2m'] = Tref - month_ds['t2m']
    else: # type == cooling
        if Tref != None: Tref = 295.15 # (22 C)
        month_ds['t2m'] = month_ds['t2m'] - Tref
    month_ds = np.maximum(month_ds, 0)

    # note: integrate sets each hour as unit 1, must divide by total number of hours (24 in a day) to obtain cumulative degree days
    dd = month_ds.integrate('time',datetime_unit='h') /24 
    return dd['t2m'].values

def HDD_UKMet_func(Tmax, Tmin, Tavg, Tref = 288.65):
    """
    Function used to calculate heating degree days in method developed by the UK Meteorological Office in 1928, intended 
    for use when only the max, min, and mean daily temperatures are known. For further details see for example
        Day, & Karayiannis, T. G. (1998). Degree-days: Comparison of calculation methods. 
        Building Services Engineering Research & Technology, 19(1), 7-13. https://doi.org/10.1177/014362449801900102
    or
        Spinoni, Vogt, J., & Barbosa, P. (2015). European degree-day climatologies and trends for the period 1951-2011. 
        International Journal of Climatology, 35(1), 25-36. https://doi.org/10.1002/joc.3959
    INPUTS:
        Tmax (float): Maximum temperature
        Tmin (float): Minimum temperature
        Tavg (float): Average temperature
        Tref (float, optional): reference or base temperature. Default is 15.5C for HDD (standard value)
    OUTPUTS:
        val (float): Degree day for the given day
    """
    if Tmax != Tmin:
        assert ((Tmax > Tavg) and (Tavg > Tmin)), \
        'HDD_UKMet_func: Should have Tmax > Tavg > Tmin. {0} {1} {2}'.format(Tmax, Tmin, Tavg)
    else:
        assert (Tmax == Tavg), \
        'HDD_UKMet_func: Tmax, Tmin, Tavg are inconsistent. {0} {1} {2}'.format(Tmax, Tmin, Tavg)
        print('WARNING HDD_UKMet_func: Tmax = Tmin, indicating constant daily temperature. Is this correct?')
    if Tref >= Tmax:
        val = Tref - Tavg
    elif (Tavg <= Tref): # implicitly includes (Tref < Tmax)
            val = (Tref - Tmin)/2 - (Tmax - Tref)/4
    elif (Tmin <= Tref): # implicitly includes (Tref < Tavg < Tmax)
        val = (Tref - Tmin)/4
    elif (Tref <= Tmin):
        val = 0
    else:
        raise Exception('HDD_UKMet_func: something went wrong, should not get here')
    return val

def CDD_UKMet_func(Tmax, Tmin, Tavg, Tref = 295.15):
    """
    Function used to calculate cooling degree days in method developed by the UK Meteorological Office in 1928, intended 
    for use when only the max, min, and mean daily temperatures are known. For further details see for example
        Day, & Karayiannis, T. G. (1998). Degree-days: Comparison of calculation methods. 
        Building Services Engineering Research & Technology, 19(1), 7-13. https://doi.org/10.1177/014362449801900102
    or
        Spinoni, Vogt, J., & Barbosa, P. (2015). European degree-day climatologies and trends for the period 1951-2011. 
        International Journal of Climatology, 35(1), 25-36. https://doi.org/10.1002/joc.3959
    INPUTS:
        Tmax (float): Maximum temperature
        Tmin (float): Minimum temperature
        Tavg (float): Average temperature
        Tref (float, optional): reference or base temperature. Default is 22C for CDD (standard value)
    OUTPUTS:
        val (float): Degree day for the given day
    """
    if Tmax != Tmin:
        assert ((Tmax > Tavg) and (Tavg > Tmin)), \
        'CDD_UKMet_func: Should have Tmax > Tavg > Tmin. {0} {1} {2}'.format(Tmax, Tmin, Tavg)
    else:
        assert (Tmax == Tavg), \
        'CDD_UKMet_func: Tmax, Tmin, Tavg are inconsistent. {0} {1} {2}'.format(Tmax, Tmin, Tavg)
        print('WARNING CDD_UKMet_func: Tmax = Tmin, indicating constant daily temperature. Is this correct?')
    if Tref >= Tmax:
        val = 0
    elif (Tavg <= Tref): # implicitly includes (Tref < Tmax)
        val = (Tmax - Tref)/4
    elif (Tmin <= Tref): # implicitly includes (Tref < Tavg < Tmax)
        val = (Tmax - Tref)/2 - (Tref - Tmin)/4
    elif (Tref <= Tmin):
        val = Tavg - Tref
    else:
        raise Exception('CDD_UKMet_func: something went wrong, should not get here')
    return val

# Create vetorized functions of HDD_UKMet_func and CDD_UKMet_func to call in 
HDD_UKMet_vecfunc = np.vectorize(HDD_UKMet_func)
CDD_UKMet_vecfunc = np.vectorize(CDD_UKMet_func)

def calc_degreedays_UKMet(type, date_str, Tref = None):
    """
    Calculate the degree days for a given month using the approach developed by the UK Meteorological Office in 1928, 
    intended for use when only the max, min, and mean daily temperatures are known. For further details see for example
        Day, & Karayiannis, T. G. (1998). Degree-days: Comparison of calculation methods. 
        Building Services Engineering Research & Technology, 19(1), 7-13. https://doi.org/10.1177/014362449801900102
    or
        Spinoni, Vogt, J., & Barbosa, P. (2015). European degree-day climatologies and trends for the period 1951-2011. 
        International Journal of Climatology, 35(1), 25-36. https://doi.org/10.1002/joc.3959
    INPUTS:
        type (str): either 'heating' for HDD or 'cooling' for CDD
        date_str (str): should be 'year-month', ex. '2020-02'
        Tref (float, optional): reference or base temperature. Default is 15.5C for HDD, 22C for CDD (standard values)
    OUTPUTS:
        dd.values (float): Cumulative degree days for the given month
    """
    if type not in ('heating','cooling'):
        print("WARNING calc_degreedays_UKMet: degree days type not understood. Defaulting to type='heating'.")
        type = 'heating'

    ds = get_temp_series() #TODO specify location here
    # this takes from midnight of first day to 11pm of last day of month
    month_ds = ds.sel(time=slice(*get_month_endpoints(date_str, include_midnight=False)))
    max_temps = month_ds.resample(time='1D').max() # get max, min, and mean daily temperatures
    min_temps = month_ds.resample(time='1D').min()
    mean_temps = (max_temps + min_temps) / 2    

    if type == 'heating':
        if Tref == None: Tref = 288.65 # (15.5 C)
        dd = np.sum(xr.apply_ufunc(HDD_UKMet_vecfunc, max_temps['t2m'], min_temps['t2m'], mean_temps['t2m'], Tref))
    else: # type == cooling
        if Tref == None: Tref = 295.15  # (22 C)
        dd = np.sum(xr.apply_ufunc(CDD_UKMet_vecfunc, max_temps['t2m'], min_temps['t2m'], mean_temps['t2m'], Tref))
    
    return dd.values

def calc_degreedays_UKMet_alternative(type, date_str, Tref = None):
    """
    Alternate implemenation of calc_degreedays_UKMet using numpy arrays. Although not usefule now, it can perhaps
    be usefull later on if the code is restructured or we use a different data source, so we choose to keep it.
    """
    if type not in ('heating','cooling'):
        print("WARNING calc_degreedays_UKMet_alternative: degree days type not understood. Defaulting to type='heating'.")
        type = 'heating'

    ds = get_temp_series() #TODO specify location here
    # this takes from midnight of first day to 11pm of last day of month
    month_ds = ds.sel(time=slice(*get_month_endpoints(date_str, include_midnight=False)))
    max_temps = month_ds.resample(time='1D').max()['t2m'].values # get max, min, and mean daily temperatures
    min_temps = month_ds.resample(time='1D').min()['t2m'].values
    mean_temps = (max_temps + min_temps) / 2    

    if type == 'heating':
        if Tref == None: Tref = 288.65 # (15.5 C)
        dd = HDD_UKMet_vecfunc(max_temps, min_temps, mean_temps, Tref)
    else: # type == cooling
        if Tref == None: Tref = 295.15  # (22 C)
        dd = CDD_UKMet_vecfunc(max_temps, min_temps, mean_temps, Tref)

    return np.sum(dd)


    


In [50]:
print(calc_degreedays_int('heating','2020-03',Tref=288.65))
print(calc_degreedays_UKMet('heating','2020-03',Tref=288.65))
print(calc_degreedays_UKMet_alternative('heating','2020-03',Tref=288.65))

285.09101060231455
282.0229671478265
282.0229671478265


In [40]:
%timeit calc_degreedays_int('heating','2020-05',Tref=288.65)
%timeit calc_degreedays_UKMet('heating','2020-05',Tref=288.65)
%timeit calc_degreedays_UKMet_alternative('heating','2020-05',Tref=288.65)

622 ms ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
695 ms ± 48.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
671 ms ± 18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


TODO:
- specify location
- make a lookup table so that if location and month has already been done, simply pull that. Otherwise do the calculation.